In [6]:
def accumulator_simd(accm_op, variables, func="_mm256_add_pd", indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        return variables[0]

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} ={func}({variables_[i]}, {variables_[i+1]});")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_simd(accm_op, variables_next_, indent_level = indent_level)

for unroll in range(1, 8+1, 1):
    with open(f"./small_dgemv_naive_c_implementation_ver2-2_YmmNaive_Unroll-OuterLoop-{unroll}.c", "w") as p:
        p.write("#include <stddef.h>\n")
        p.write("#include <stdio.h>\n")
        p.write("#include <stdlib.h>\n")
        p.write("#include <stdint.h>\n")
        p.write("#include <math.h>\n")
        p.write("#include <omp.h>\n")
        p.write("#include <immintrin.h>\n") 
        p.write("#include <xmmintrin.h>\n")
        
        p.write(f"""
double sum_ymm_elements_v2_2_{unroll}(__m256d ymmX){{
    __m128d xmm_upper = _mm256_extractf128_pd(ymmX, 1);
    __m128d xmm_lower = _mm256_extractf128_pd(ymmX, 0);

    __m128d xmm_sum = _mm_add_pd(xmm_upper, xmm_lower);

    __m128d xmm_high_low = _mm_add_pd(xmm_sum, _mm_unpackhi_pd(xmm_sum, xmm_sum));

    double result;
    _mm_store_sd(&result, xmm_high_low);

    return result;
}}        
        """)
        
        # Normal ----------------------------------------------------------------------------------------------------------
        y_start, y_end = 1, 1+unroll
        a_start, a_end = 1+unroll, 1+unroll*2

        remain_op_i = "".join([f"""
            y[i] += a[i + (k+{j})*lda] * x[k+{j}];""" for j in range(unroll)])
            
        load_x = "".join([f"""
        ymm{i} = _mm256_set1_pd(x[k+{i}]);""" for i in range(unroll)])
        
        load_y = f"""
            ymm{unroll+0} = _mm256_loadu_pd(&y[i]);"""
        
        load_a = "".join([f"""
            ymm{i+unroll+1} = _mm256_loadu_pd(&a[i+(k+{i})*lda]);""" for i in range(unroll)])
        
        mul_ax = "".join([f"""
            ymm{i+unroll+1} = _mm256_mul_pd(ymm{i+unroll+1}, ymm{i});""" for i in range(unroll)])
        
        accm_op = []
        variables = [f"ymm{unroll+0}"] + [f"ymm{i+unroll+1}" for i in range(unroll)]
        accumulator_simd(accm_op, variables, indent_level=3)
        accm_op = "\n".join(accm_op)

        code = f"""
void mydgemv_n_ver2_2_unroll{unroll}(double a[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    __m256d ymm0, {", ".join([f"ymm{i}" for i in range(1, 2+unroll*2)])};
    double tmp_x;
    for (int64_t i=0; i<lda; i++){{
        y[i] = 0.0;
    }}

    int64_t k_remain = ldx % {unroll}, i_remain = lda % 4;
    for (int64_t k=0; k<ldx-k_remain; k+={unroll}){{
        {load_x}
        for (int64_t i=0; i<lda-i_remain; i+=4){{
            {load_y}
            {load_a}
            {mul_ax}
            
            {accm_op}
            _mm256_storeu_pd(&y[i], ymm{unroll});
        }}
        for (int64_t i = lda-i_remain; i<lda; i++){{
            {remain_op_i}
        }}
    }}
    
    for (int64_t k=ldx-k_remain; k<ldx; k++){{
        ymm0 = _mm256_set1_pd(x[k]);        
        for (int64_t i=0; i<lda-i_remain; i+=4){{
            ymm1 = _mm256_loadu_pd(&y[i]);
            ymm2 = _mm256_loadu_pd(&a[i+k*lda]);
            
            ymm2 = _mm256_mul_pd(ymm0, ymm2);
            
            ymm1 = _mm256_add_pd(ymm1, ymm2);
            
            _mm256_storeu_pd(&y[i], ymm1);
        }}
        
        for (int64_t i = lda-i_remain; i<lda; i++){{
            y[i] += a[i+k*lda] * x[k+0];
        }}         
    }}
}}
        """
        p.write(code)

        # Transposed ----------------------------------------------------------------------------------------------------------
        remain_op_i = f"""
    for (int64_t i=lda-i_remain; i<lda; i++){{
        double tmp=0.0;
        for (int64_t k=0; k<ldx-k_remain; k++){{
            tmp += a_t[k+i*ldx] * x[k];
        }}
        y[i] = tmp;
    }}
        """
            
        init_y = "".join([f"""ymm{i} = _mm256_setzero_pd();
        """ for i in range(unroll)])
        
        load_x = f"ymm{unroll} = _mm256_loadu_pd(&x[k]);"
        
        load_a = "".join([f"""ymm{i+unroll+1} = _mm256_loadu_pd(&a_t[k+(i+{i})*ldx]);
            """ for i in range(unroll)])
        
        mul_ax = "".join([f"""ymm{i+unroll+1} = _mm256_mul_pd(ymm{unroll}, ymm{i+unroll+1});
            """ for i in range(unroll)])
        
        add_ax2y = "".join([f"""ymm{i} = _mm256_add_pd(ymm{i}, ymm{i+unroll+1});
            """ for i in range(unroll)])
        
        store_y = "".join([f"""y[i+{i}] = sum_ymm_elements_v2_2_{unroll}(ymm{i});
        """ for i in range(unroll)])
        
        code = f"""
void mydgemv_t_ver2_2_unroll{unroll}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    __m256d ymm0, {", ".join([f"ymm{i}" for i in range(1, 1+unroll*2)])};

    int64_t k_remain = ldx % 4, i_remain = lda % {unroll};
    for (int64_t i=0; i<lda-i_remain; i+={unroll}){{
        {init_y}
        for (int64_t k=0; k<ldx-k_remain; k+=4){{
            {load_x}
            
            {load_a}
            {mul_ax}
            {add_ax2y}
        }}
        {store_y}
    }}
    {remain_op_i}
}}
        """
        p.write(code)
